VERSIONE FINALE

Minimizzare sia per il numero di gruppi da creare sia per il costo delle chiamate api, il cui costo è pari a 1 per ogni gruppo e pari a 1 ogni 100 ip, sempre con il vincolo che al massimo in un gruppo ci possono essere 1000 ip. Vresione refactored 1/10

In [1]:
import pulp
import json

# 1.0 Data Retrieval

Dati forniti da Lu.

In [2]:
res = {"query": "services.http.response.html_title: \"X Acceleration Codec\" or services.http.response.body: \"X Acceleration Codec\"","field": "location.country","total": 2194,"duration": 1099,"total_omitted": 0,"potential_deviation": 0,"buckets": [{"key": "Netherlands","count": 454},{"key": "Spain","count": 392},{"key": "Germany","count": 267},{"key": "France","count": 246},{"key": "United States","count": 139},{"key": "Brazil","count": 108},{"key": "United Kingdom","count": 59},{"key": "Finland","count": 55},{"key": "Bulgaria","count": 41},{"key": "Italy","count": 39},{"key": "Romania","count": 32},{"key": "Ukraine","count": 32},{"key": "Canada","count": 25},{"key": "Serbia","count": 23},{"key": "Bosnia and Herzegovina","count": 21},{"key": "Thailand","count": 19},{"key": "India","count": 18},{"key": "Turkey","count": 16},{"key": "Singapore","count": 15},{"key": "Colombia","count": 14},{"key": "Pakistan","count": 14},{"key": "Poland","count": 14},{"key": "Lithuania","count": 12},{"key": "Israel","count": 10},{"key": "Portugal","count": 10},{"key": "Iraq","count": 9},{"key": "Russia","count": 8},{"key": "Croatia","count": 7},{"key": "Sweden","count": 7},{"key": "Austria","count": 6},{"key": "Switzerland","count": 6},{"key": "Indonesia","count": 5},{"key": "Belgium","count": 4},{"key": "Costa Rica","count": 4},{"key": "Palestinian Territory","count": 4},{"key": "Albania","count": 3},{"key": "Australia","count": 3},{"key": "Czech Republic","count": 3},{"key": "Estonia","count": 3},{"key": "Macedonia","count": 3},{"key": "Peru","count": 3},{"key": "South Africa","count": 3},{"key": "Tunisia","count": 3},{"key": "Argentina","count": 2},{"key": "Cyprus","count": 2},{"key": "Dominican Republic","count": 2},{"key": "Hungary","count": 2},{"key": "Iceland","count": 2},{"key": "Ireland","count": 2},{"key": "Paraguay","count": 2},{"key": "United Arab Emirates","count": 2},{"key": "Armenia","count": 1},{"key": "Bolivia","count": 1},{"key": "Chile","count": 1},{"key": "Denmark","count": 1},{"key": "Ecuador","count": 1},{"key": "Egypt","count": 1},{"key": "El Salvador","count": 1},{"key": "French Polynesia","count": 1},{"key": "Honduras","count": 1},{"key": "Japan","count": 1},{"key": "Jordan","count": 1},{"key": "Malta","count": 1},{"key": "Montenegro","count": 1},{"key": "Morocco","count": 1},{"key": "Myanmar","count": 1},{"key": "Nepal","count": 1},{"key": "Norway","count": 1},{"key": "Slovakia","count": 1},{"key": "Vietnam","count": 1}]}

In [3]:
data = res['buckets']

# 2.0 Modeling

## 2.1 Business constraints definition

In [4]:
capacity = 1000 # max num ip per gruppo
countries = [item['key'] for item in data]
counts = [item['count'] for item in data]
num_items = len(countries)
num_bins = num_items  # limite superiore: al mamissimo tanti gruppi quanti paesi ci sono

## 2.2 Mathematical definition of the problem

In [5]:
prob = pulp.LpProblem("Minimizza il costo delle chiamate API a Censys Free", pulp.LpMinimize) # definisco il problema e la tipologia di ottimizzazione

/opt/conda/lib/python3.10/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### 2.2.1 Decision variables

In [6]:
x = pulp.LpVariable.dicts('x',((i, j) for i in range(num_items) for j in range(num_bins)),cat='Binary') # il paese i è assegnato al gruppo j?
y = pulp.LpVariable.dicts('y',(j for j in range(num_bins)),cat='Binary') # il gruppo j è utilizzato?
s = pulp.LpVariable.dicts('s',(j for j in range(num_bins)),lowBound=0,cat='Continuous') # per contare il totale degli indirizzi ip in ciascun gruppo. Continua xk rende i calcoli più veloci
c = pulp.LpVariable.dicts('c',(j for j in range(num_bins)),lowBound=0,cat='Integer') # per contare il numero di blocchi da 100 ip in ciascun gruppo

### 2.2.2 Objective function: cosa vogliamo miminizzare? il costo!

In [7]:
prob += pulp.lpSum(y[j] + c[j] for j in range(num_bins)) # semplice funzione di costo, data dalla somma del costo dei gruppi + il costo ogni 100 ip

### 2.2.3 Constraints

In [8]:
# 1) vincolo per verificare che ogni paese è assegnato ad un solo gruppo
for i in range(num_items):
    prob += pulp.lpSum(x[(i, j)] for j in range(num_bins)) == 1

In [9]:
# 2) vincolo per verificare che il numero di ip in un gruppo non superi il limite di 1000. Prima conto gli ip per ogni gruppo, poi verifico che sia inferiore al massimo consentito
for j in range(num_bins):
    prob += s[j] == pulp.lpSum(counts[i] * x[(i, j)] for i in range(num_items))
    prob += s[j] <= capacity * y[j]

In [10]:
# 3) vincolo per verificare quale gruppo è utilizzato in base al fatto se contiene o meno dei paesi
for i in range(num_items):
    for j in range(num_bins):
        prob += x[(i, j)] <= y[j]

In [11]:
# 4) vincolo per verificare quanti gruppi fare in base al numero di ip e il conseguente costo
for j in range(num_bins):
    prob += s[j] <= 100 * c[j]
    prob += s[j] >= 100 * (c[j] - 1) + 1 - (1 - y[j]) * capacity

In [12]:
# 5) vincolo per verificare che non ci siano gruppi vuoti utilizzati
for j in range(num_bins):
    prob += c[j] <= (capacity // 100) * y[j]

In [13]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/35d6a598957a43649aefe4ca58bd55ea-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/35d6a598957a43649aefe4ca58bd55ea-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5325 COLUMNS
At line 35846 RHS
At line 41167 BOUNDS
At line 46208 ENDATA
Problem MODEL has 5320 rows, 5110 columns and 20300 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 24.134 - 0.02 seconds
Cgl0003I 0 fixed, 70 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 5320 rows, 5110 columns (5040 integer (4970 of which binary)) and 20300 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 33 integers unsatisfied sum - 6.4882
Cbc0038I Pass   1: (0.20 seconds) suminf.    1.2

1

# 3.0 Output

In [14]:
solver_status = pulp.LpStatus[prob.status]

Struttura json finale

In [15]:
output_data = {
    'stato_solver': solver_status,
    'totale_gruppi': 0,
    'totale_spesa': 0,
    'gruppi': []
}

contatore per partire da 1 e non da 0 

In [16]:
group_number = 1

Ciclo for per capire quali gruppi sono stati utilizzati di tutti i possibili (derivati del numero di paesi) e analogamente per ciclare su tutti i paesi e vedere quale paese c'è all'interno di ciascun gruppo. Uso 0.5 invece che 1 xk mi dava problemi con gli arrotondamenti (0.99999999)

In [17]:
for j in range(num_bins):
    if y[j].varValue > 0.5:
        total_ip = 0
        countries_in_group = []
        for i in range(num_items):
            if x[(i, j)].varValue > 0.5:
                country_info = {
                    'paese': countries[i],
                    'numero_ip': counts[i]
                }
                countries_in_group.append(country_info)
                total_ip += counts[i]

        if total_ip > 0:
            group_cost = 1 
            ip_cost = int(c[j].varValue) 
            total_group_cost = group_cost + ip_cost 

            output_data['totale_gruppi'] += 1
            output_data['totale_spesa'] += total_group_cost

            group_info = {
                'numero_gruppo': group_number,
                'totale_ip': total_ip,
                'costo_ip': ip_cost,
                'costo_gruppo': group_cost,
                'costo_totale_gruppo': total_group_cost,
                'lista_paesi': countries_in_group
            }

            output_data['gruppi'].append(group_info)
            group_number += 1

In [18]:
output_json = json.dumps(output_data, indent=2)

In [19]:
print(output_json)

{
  "stato_solver": "Optimal",
  "totale_gruppi": 3,
  "totale_spesa": 25,
  "gruppi": [
    {
      "numero_gruppo": 1,
      "totale_ip": 196,
      "costo_ip": 2,
      "costo_gruppo": 1,
      "costo_totale_gruppo": 3,
      "lista_paesi": [
        {
          "paese": "Brazil",
          "numero_ip": 108
        },
        {
          "paese": "Finland",
          "numero_ip": 55
        },
        {
          "paese": "India",
          "numero_ip": 18
        },
        {
          "paese": "Colombia",
          "numero_ip": 14
        },
        {
          "paese": "Nepal",
          "numero_ip": 1
        }
      ]
    },
    {
      "numero_gruppo": 2,
      "totale_ip": 998,
      "costo_ip": 10,
      "costo_gruppo": 1,
      "costo_totale_gruppo": 11,
      "lista_paesi": [
        {
          "paese": "Spain",
          "numero_ip": 392
        },
        {
          "paese": "Germany",
          "numero_ip": 267
        },
        {
          "paese": "United States",
